In [1]:
try:
    import google.colab  # noqa: F401

    # specify the version of DataEval (==X.XX.X) for versions other than the latest
    %pip install -q dataeval[torch]
except Exception:
    pass

In [2]:
from functools import partial

import numpy as np
import torch

from dataeval.detectors.drift import (
    DriftCVM,
    DriftKS,
    DriftMMD,
    preprocess_drift,
)
from dataeval.utils.torch.datasets import MNIST
from dataeval.utils.torch.models import AriaAutoencoder

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# Load in the training mnist dataset and use the first 4000
train_ds = MNIST(root="./data/", train=True, download=True, size=4000, dtype=np.float32, channels="channels_first")

# Split out the images and labels
images, labels = train_ds.data, train_ds.targets

Files already downloaded and verified


In [4]:
print("Number of samples: ", len(images))
print("Image shape:", images[0].shape)

Number of samples:  4000
Image shape: (1, 28, 28)


In [5]:
data_reference = images[0:2000]
data_control = images[2000:]

In [6]:
# define encoder
encoder_net = AriaAutoencoder(1).encoder.to(device)

# define preprocessing function
preprocess_fn = partial(preprocess_drift, model=encoder_net, batch_size=64, device=device)

# initialise drift detectors
detectors = [detector(data_reference, preprocess_fn=preprocess_fn) for detector in [DriftMMD, DriftCVM, DriftKS]]

In [7]:
[(type(detector).__name__, detector.predict(data_control).is_drift) for detector in detectors]

[('DriftMMD', False), ('DriftCVM', False), ('DriftKS', False)]

In [8]:
corruption = MNIST(
    root="./data",
    train=True,
    download=False,
    size=2000,
    dtype=np.float32,
    channels="channels_first",
    corruption="translate",
)
corrupted_images = corruption.data

Files already downloaded and verified


In [9]:
print("Number of corrupted samples: ", len(corrupted_images))
print("Corrupted image shape:", corrupted_images[0].shape)

Number of corrupted samples:  2000
Corrupted image shape: (1, 28, 28)


In [10]:
[(type(detector).__name__, detector.predict(corrupted_images).is_drift) for detector in detectors]

[('DriftMMD', True), ('DriftCVM', True), ('DriftKS', True)]